# MINERVA Model Training

**Strategic Pattern Analysis Specialist**

This notebook trains the MINERVA model, which specializes in:
- Pattern recognition and analysis
- Strategic reasoning
- Global pattern understanding
- Grid attention mechanisms
- Object relationship analysis

## Setup Environment

In [ ]:
# Clone repository and install dependencies
!git clone https://github.com/AutomataControls/AutomataNexus_Olympus_AGI2.git /content/AutomataNexus_Olympus_AGI2
!cd /content/AutomataNexus_Olympus_AGI2 && pip install -r requirements.txt -q

In [ ]:
# Verify environment
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## Start MINERVA Training

Training configuration:
- **Architecture**: Grid Attention + Relational Networks
- **Parameters**: ~2.1M
- **Batch Size**: 512 (effective 2048 with gradient accumulation)
- **Epochs**: 300
- **Learning Rate**: 0.005 with ReduceLROnPlateau

In [ ]:
# Train MINERVA model
!cd /content/AutomataNexus_Olympus_AGI2 && python scripts/training/train_minerva.py

## 📊 Monitor Training Progress

In [ ]:
# Initialize MINERVA Iterative Training Controller
import json
import os
import datetime
from typing import Dict, List

class MinervaIterativeTrainer:
    def __init__(self, base_dir='/content/AutomataNexus_Olympus_AGI2'):
        self.base_dir = base_dir
        self.iteration_log_file = f"{base_dir}/arc_models_v4/minerva_iterations.json"
        self.current_params = self.load_default_params()
        self.iterations = self.load_iteration_history()
    
    def load_default_params(self) -> Dict:
        return {
            'learning_rate': 0.005,
            'batch_size': 512,
            'epochs_per_stage': 100,
            'transformation_penalty': 0.5,
            'exact_match_bonus': 5.0,
            'gradient_accumulation_steps': 4,
            'use_mept': True,
            'use_leap': True,
            'use_prism': True,
            'target_accuracy': 95.0,
            'exact_injection_epochs': 100,
            'stage_lr_reset': True,
            'adaptive_lr': True,
            'checkpoint_reset_threshold': 2.0
        }
    
    def load_iteration_history(self) -> List[Dict]:
        if os.path.exists(self.iteration_log_file):
            with open(self.iteration_log_file, 'r') as f:
                return json.load(f)
        return []
    
    def save_iteration_history(self):
        os.makedirs(os.path.dirname(self.iteration_log_file), exist_ok=True)
        with open(self.iteration_log_file, 'w') as f:
            json.dump(self.iterations, f, indent=2)
    
    def log_iteration(self, params: Dict, results: Dict):
        iteration = {
            'iteration': len(self.iterations) + 1,
            'timestamp': datetime.datetime.now().isoformat(),
            'parameters': params.copy(),
            'results': results.copy()
        }
        self.iterations.append(iteration)
        self.save_iteration_history()
    
    def get_best_iteration(self) -> Dict:
        if not self.iterations:
            return None
        return max(self.iterations, key=lambda x: x['results'].get('best_exact', 0))
    
    def suggest_next_params(self) -> Dict:
        if len(self.iterations) < 2:
            return self.current_params.copy()
        
        best = self.get_best_iteration()
        latest = self.iterations[-1]
        
        suggestions = best['parameters'].copy() if best else self.current_params.copy()
        
        if latest and best:
            latest_exact = latest['results'].get('best_exact', 0)
            best_exact = best['results'].get('best_exact', 0)
            
            if latest_exact < 1.0:
                suggestions['learning_rate'] = min(0.008, suggestions['learning_rate'] * 1.3)
                suggestions['adaptive_lr'] = True
                suggestions['checkpoint_reset_threshold'] = 2.0
            elif latest_exact < best_exact * 0.8:
                suggestions['learning_rate'] *= 0.7
                suggestions['transformation_penalty'] = max(0.5, suggestions['transformation_penalty'])
            elif latest_exact > best_exact * 1.1:
                suggestions['learning_rate'] = min(0.01, suggestions['learning_rate'] * 1.1)
        
        return suggestions
    
    def display_history(self):
        if not self.iterations:
            print("No MINERVA iterations found. Starting fresh!")
            return
        
        print("MINERVA Training History:")
        print("-" * 85)
        for i, iteration in enumerate(self.iterations):
            exact = iteration['results'].get('best_exact', 0)
            loss = iteration['results'].get('best_val_loss', float('inf'))
            lr = iteration['parameters'].get('learning_rate', 0)
            trans_pen = iteration['parameters'].get('transformation_penalty', 0)
            exact_bonus = iteration['parameters'].get('exact_match_bonus', 0)
            timestamp = iteration['timestamp'][:16]
            
            status = "BEST" if iteration == self.get_best_iteration() else ""
            print(f"{status} Iter {i+1}: {exact:.2f}% exact | Loss: {loss:.4f} | LR: {lr:.4f} | Trans: {trans_pen:.1f} | Bonus: {exact_bonus:.1f} | {timestamp}")
        
        print("-" * 85)
        best = self.get_best_iteration()
        if best:
            print(f"Best: Iteration {best['iteration']} with {best['results']['best_exact']:.2f}% exact match")

minerva_trainer = MinervaIterativeTrainer()
suggested_params = minerva_trainer.suggest_next_params()
minerva_trainer.display_history()
print("\nMINERVA trainer initialized. Ready for training.")

In [ ]:
# Check GPU usage and training progress
import subprocess
import os
import glob
import time

# Show GPU status
gpu_info = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print("GPU Status:")
print(gpu_info.stdout)

# Check for MINERVA checkpoints
checkpoints = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_*.pt')
if checkpoints:
    print("\nMINERVA Checkpoints:")
    for checkpoint in sorted(checkpoints):
        size = os.path.getsize(checkpoint) / (1024**2)  # Size in MB
        mtime = os.path.getmtime(checkpoint)
        print(f"  {os.path.basename(checkpoint)}: {size:.1f} MB - {time.ctime(mtime)}")

In [ ]:
# MINERVA Iterative Training
import subprocess
import tempfile
import os

if 'minerva_trainer' not in globals():
    print("Error: Run the trainer initialization cell first!")
    raise NameError("minerva_trainer not defined")

LEARNING_RATE = suggested_params.get('learning_rate', 0.005)
BATCH_SIZE = suggested_params.get('batch_size', 512)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 100)
TRANSFORMATION_PENALTY = 0.5
EXACT_MATCH_BONUS = 5.0
GRADIENT_ACCUMULATION_STEPS = suggested_params.get('gradient_accumulation_steps', 4)
USE_MEPT = True
USE_LEAP = True
USE_PRISM = True
TARGET_ACCURACY = 95.0

print(f"Starting MINERVA training iteration {len(minerva_trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Features: MEPT={USE_MEPT}, LEAP={USE_LEAP}, PRISM={USE_PRISM}")
print(f"Target accuracy: {TARGET_ACCURACY}%")

modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

import scripts.training.train_minerva as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.USE_MEPT = {USE_MEPT}
train_module.USE_LEAP = {USE_LEAP}
train_module.USE_PRISM = {USE_PRISM}

try:
    import scripts.training.stage0_exact_match_boost as boost_module
    original_inject = boost_module.inject_exact_match_training
    def inject_with_target(*args, **kwargs):
        kwargs['target_accuracy'] = {TARGET_ACCURACY}
        kwargs['num_epochs'] = 100
        return original_inject(*args, **kwargs)
    boost_module.inject_exact_match_training = inject_with_target
except ImportError:
    print("Warning: stage0_exact_match_boost not available")

train_module.train_minerva()
"""

with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    print("MINERVA training timeout: 8 hours")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=28800
    )
    
    print("MINERVA training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    if os.path.exists(script_path):
        os.unlink(script_path)

In [ ]:
# Log Results and Update History
import glob
import torch
import json

current_params = {
    'learning_rate': LEARNING_RATE,
    'batch_size': BATCH_SIZE,
    'epochs_per_stage': EPOCHS_PER_STAGE,
    'transformation_penalty': TRANSFORMATION_PENALTY,
    'exact_match_bonus': EXACT_MATCH_BONUS,
    'gradient_accumulation_steps': GRADIENT_ACCUMULATION_STEPS,
    'use_mept': USE_MEPT,
    'use_leap': USE_LEAP,
    'use_prism': USE_PRISM
}

model_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_best.pt')
results = {}

if model_files:
    checkpoint = torch.load(model_files[0], map_location='cpu')
    results = {
        'best_exact': checkpoint.get('val_exact', 0),
        'best_val_loss': checkpoint.get('val_loss', float('inf')),
        'training_epoch': checkpoint.get('epoch', 0),
        'stage': checkpoint.get('stage', 0)
    }
    
    print(f"Training completed!")
    print(f"   Best exact match: {results['best_exact']:.2f}%")
    print(f"   Best validation loss: {results['best_val_loss']:.4f}")
    print(f"   Training epochs: {results['training_epoch']}")
    
    trainer.log_iteration(current_params, results)
    
    print("\n" + "="*60)
    trainer.display_history()
    
    print("\nSuggestions for next iteration:")
    next_params = trainer.suggest_next_params()
    
    current_exact = results['best_exact']
    best_iteration = trainer.get_best_iteration()
    best_exact = best_iteration['results']['best_exact'] if best_iteration else 0
    
    if current_exact >= best_exact * 0.95:
        print("   Performance is good! Try:")
        if current_exact > best_exact:
            print(f"   New best! Consider fine-tuning with LR={next_params['learning_rate']:.4f}")
        else:
            print(f"   Try slightly different parameters or increase training epochs")
    else:
        print("   Performance below expectations. Suggestions:")
        print(f"   Reduce learning rate to {next_params['learning_rate']:.4f}")
        print(f"   Increase exact match bonus to {next_params['exact_match_bonus']:.1f}")
        print("   Consider enabling all features (MEPT, LEAP, PRISM)")
    
else:
    print("No model checkpoint found. Training may have failed.")
    
print("\nTo continue training, modify parameters above and re-run the training cell.")

## Iterative Training & Hyperparameter Tuning

**Enhanced Training Loop with Checkpoint Resuming**

This section allows you to:
- Resume training from checkpoints
- Adjust hyperparameters between iterations
- Track performance improvements
- Get automated suggestions for next iteration

In [ ]:
# Load and evaluate MINERVA model
import torch
import json
import glob

model_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_best.pt')

if model_files:
    model_file = model_files[0]
    checkpoint = torch.load(model_file, map_location='cpu')
    
    print("MINERVA Training Results:")
    print(f"  Best Validation Exact Match: {checkpoint.get('val_exact', 'N/A'):.2f}%")
    print(f"  Training Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"  Stage: {checkpoint.get('stage', 'N/A')}")
    print(f"  Model Size: {os.path.getsize(model_file) / (1024**2):.1f} MB")
else:
    print("MINERVA model not found. Training may still be in progress.")

report_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_training_report_*.json')
if report_files:
    latest_report = sorted(report_files)[-1]
    with open(latest_report, 'r') as f:
        report = json.load(f)
    print(f"\nLatest Training Report:")
    print(f"  Best Exact Match: {report.get('best_exact', 'N/A'):.2f}%")
    print(f"  Final Validation Loss: {report.get('best_val_loss', 'N/A'):.4f}")
    print(f"  Training Duration: {report.get('training_time', 'N/A')}")

## Troubleshooting

If you encounter issues:

In [ ]:
# Clear GPU memory if needed
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
print(f"Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1e9:.2f} GB")

---

## Training Complete!

Your MINERVA model has been trained and saved to `/content/AutomataNexus_Olympus_AGI2/arc_models_v4/`

**Next Steps:**
- Train other OLYMPUS models (ATLAS, IRIS, CHRONOS, PROMETHEUS)
- Use the ensemble training notebook for combined training
- Test the model on ARC evaluation tasks

**AutomataNexus OLYMPUS AGI2** - *Where Neural Networks Meet Symbolic Logic*